In [1]:
import os, torch
from pypdf import PdfReader
import chromadb
from sentence_transformers import SentenceTransformer

from docx import Document
from pptx import Presentation


# ================================
# 1. Load file (PDF, DOCX, PPTX)
# ================================
def load_file_pages(path):
    ext = os.path.splitext(path)[1].lower()

    # ========== PDF ==========
    if ext == ".pdf":
        reader = PdfReader(path)
        pages = []
        for page in reader.pages:
            text = page.extract_text() or ""
            pages.append(text)
        return pages

    # ========== DOCX ==========
    elif ext == ".docx":
        doc = Document(path)
        pages = []

        buffer = []
        paragraph_count = 0

        for para in doc.paragraphs:
            text = para.text.strip()
            if text:
                buffer.append(text)
                paragraph_count += 1

            # Cho thành "page" sau mỗi 20 đoạn (tùy chỉnh)
            if paragraph_count >= 20:
                pages.append("\n".join(buffer))
                buffer = []
                paragraph_count = 0

        if buffer:
            pages.append("\n".join(buffer))

        return pages

    # ========== PPTX ==========
    elif ext == ".pptx":
        pres = Presentation(path)
        pages = []

        for slide in pres.slides:
            slide_text = []
            for shape in slide.shapes:
                if hasattr(shape, "text"):
                    slide_text.append(shape.text)
            pages.append("\n".join(slide_text))

        return pages

    # ========== Không hỗ trợ ==========
    else:
        raise ValueError("Unsupported file format: only PDF, DOCX, PPTX.")



# ================================
# 2. Chunk 1 trang
# ================================
def chunk_page(text, chunk_size=800, overlap=200):
    words = text.split()
    chunks = []
    start = 0

    while start < len(words):
        end = start + chunk_size
        chunk = " ".join(words[start:end])
        chunks.append(chunk)
        start = end - overlap

    return chunks


# ================================
# 3. Build chunks + metadata
# ================================
def build_chunks(path):
    pages = load_file_pages(path)

    all_chunks = []
    all_ids = []
    all_meta = []

    for page_idx, text in enumerate(pages):
        page_number = page_idx + 1
        chunks = chunk_page(text)

        for ci, c in enumerate(chunks):
            all_chunks.append(c)
            all_ids.append(f"{os.path.basename(path)}_p{page_number}_c{ci}")
            all_meta.append({
                "page": page_number,
                "chunk": ci
            })

    return all_chunks, all_ids, all_meta, pages



# ================================
# 4. Model + Vector DB
# ================================
model = SentenceTransformer("Qwen/Qwen3-Embedding-0.6B")

chroma = chromadb.Client()
collection = chroma.get_or_create_collection(
    name="pdf_docs",
    metadata={"hnsw:space": "cosine"}
)



# ================================
# 5. Index PDF/DOCX/PPTX
# ================================
def index_pdf(path, batch_size=2):
    chunks, ids, metadata, pages = build_chunks(path)

    all_embeds = []

    for i in range(0, len(chunks), batch_size):
        batch = chunks[i : i + batch_size]

        with torch.no_grad():
            vec = model.encode(batch).tolist()

        all_embeds.extend(vec)

        if torch.cuda.is_available():
            torch.cuda.empty_cache()

        print(f"Đã embed {i + len(batch)}/{len(chunks)} chunks", end="\r")

    collection.add(
        ids=ids,
        documents=chunks,
        metadatas=metadata,
        embeddings=all_embeds
    )

    print(f"\nIndexed {len(chunks)} chunks từ file {path}")
    return pages



# ================================
# 6. Lấy trang lân cận
# ================================
def get_surrounding_pages(page, pages):
    prev_page = pages[page - 2] if page > 1 else None
    this_page = pages[page - 1]
    next_page = pages[page] if page < len(pages) else None

    return prev_page, this_page, next_page



# ============================================================
# 7. SEARCH 2 BƯỚC: TopK → Expand → Re-chunk → Rerank
# ============================================================
def search(query, pages, top_k_first=3, top_k_second=5, chunk_size=800, overlap=200):

    # STEP 1 ───────────────────────────────────────────────
    q_emb = model.encode([query]).tolist()

    result = collection.query(
        query_embeddings=q_emb,
        n_results=top_k_first,
        include=["documents", "metadatas"]
    )

    metas = result["metadatas"][0]

    # STEP 2 ───────────────────────────────────────────────
    expanded_pages = []
    seen = set()

    for meta in metas:
        page = meta["page"]
        prev_page, this_page, next_page = get_surrounding_pages(page, pages)
        candidates = [prev_page, this_page, next_page]

        for p in candidates:
            if p and p not in seen:
                expanded_pages.append(p)
                seen.add(p)

    # STEP 3 ───────────────────────────────────────────────
    big_text = "\n\n".join(expanded_pages)
    words = big_text.split()

    re_chunks = []
    start = 0
    while start < len(words):
        end = start + chunk_size
        chunk = " ".join(words[start:end])
        re_chunks.append(chunk)
        start = end - overlap

    # STEP 4 ───────────────────────────────────────────────
    embeds = model.encode(re_chunks).tolist()

    temp = chromadb.Client().create_collection(
        name="temp_rerank",
        metadata={"hnsw:space": "cosine"},
        get_or_create=True
    )

    temp_ids = [f"rechunk_{i}" for i in range(len(re_chunks))]
    temp.add(ids=temp_ids, embeddings=embeds, documents=re_chunks)

    result2 = temp.query(
        query_embeddings=q_emb,
        n_results=top_k_second,
        include=["documents", "distances"]
    )

    docs2 = result2["documents"][0]
    dists2 = result2["distances"][0]

    # STEP 5 ───────────────────────────────────────────────
    output = []
    for i, (doc, dist) in enumerate(zip(docs2, dists2), start=1):
        output.append({
            "rank": i,
            "matched_chunk": doc,
            "score": 1 - dist,
            "raw_distance": dist
        })

    return output



# # ============================================================
# # 8. RUN
# # ============================================================
# if __name__ == "__main__":
#     pages = index_pdf("test.docx")   # hoặc test.pdf, test.pptx

#     results = search("Mô hình nào được dùng?", pages)

#     for r in results:
#         print("="*80)
#         print("Rank:", r["rank"])
#         print("Score:", r["score"])
#         print(r["matched_chunk"])


In [2]:
pages = index_pdf("test2.docx")

Đã embed 20/20 chunks
Indexed 20 chunks từ file test2.docx


In [5]:
results = search("Điều 20. Trách nhiệm triển khai Quy chế", pages, top_k_first=3, top_k_second=5, chunk_size=300, overlap=100)

for r in results:
    print("="*80)
    print("Thông tin", r["rank"], ":")
    print("Uy tín", r["score"],":")
    print(r["matched_chunk"])

Thông tin 1 :
Uy tín 0.41750895977020264 :
chế này; các quy định của Luật Bảo vệ bí mật nhà nước và các văn bản hướng dẫn thi hành; các quy định về bảo vệ bí mật của Bộ Thông tin và Truyền thông và Ủy ban quản lý vốn nhà nước tại doanh nghiệp. 1. Tổng giám đốc có trách nhiệm hướng dẫn, đôn đốc, kiểm tra việc thực hiện Quy chế này tại các Ban, Văn phòng, Phòng thuộc khối Cơ quan Tổng công ty và các đơn vị trực thuộc Tổng công ty. 2. Trưởng các Ban, Văn phòng, Phòng thuộc khối Cơ quan Tổng công ty, Giám đốc các đơn vị trực thuộc Tổng công ty có trách nhiệm tổ chức thực hiện và phổ biến Quy chế này đến cán bộ, nhân viên thuộc quyền quản lý để thực hiện. 3. Ban Công nghệ thông tin Tổng công ty có trách nhiệm chủ trì rà soát và hướng dẫn, quy định việc mã hoá, các biện pháp đảm bảo tính bảo mật trong trường hợp sử dụng máy tính hoặc các thiết bị khác, thư điện tử, hệ thống E-Office của Tổng công ty để xử lý, lưu trữ các thông tin, tài liệu mật cơ quan. Nguyễn Thị Hiếu hieu.nth@mobifone.vn h

In [2]:
!python -m spacy download en_core_web_md


Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/33.5 MB ? eta -:--:--
     ---------------------------------------- 0.3/33.5 MB ? eta -:--:--
     -- ------------------------------------- 1.8/33.5 MB 6.7 MB/s eta 0:00:05
     ---- ----------------------------------- 3.7/33.5 MB 7.3 MB/s eta 0:00:05
     ----- ---------------------------------- 4.5/33.5 MB 6.2 MB/s eta 0:00:05
     ------ --------------------------------- 5.5/33.5 MB 6.0 MB/s eta 0:00:05
     ------- -------------------------------- 6.6/33.5 MB 6.3 MB/s eta 0:00:05
     -------- ------------------------------- 7.1/33.5 MB 5.5 MB/s eta 0:00:05
     --------- ------------------------------ 7.9/33.5 MB 5.2 MB/s eta 0:00:05
     ---------- ----------------------------- 9.2/33.5 MB 5.2 MB/s eta 0:00:05
     ----------- ---------------------------- 9.4/33.5 MB 5.2 MB/s eta 0:00:05
     ------------ --------------------------- 10.2/33.5 MB 4.7 MB/s

In [ ]:
import os, torch, re
from pypdf import PdfReader
import chromadb
from sentence_transformers import SentenceTransformer

from docx import Document
from pptx import Presentation

import spacy
import dateparser
from transformers import pipeline


# =====================================================
# NER MODULE (NEW) — Nhận diện thực thể + Highlight
# =====================================================

nlp_spacy = spacy.load("en_core_web_md")

ner_multi = pipeline(
    "ner",
    model="Babelscape/wikineural-multilingual-ner",
    grouped_entities=True
)

email_regex = r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}"
phone_regex = r"\+?\d[\d\- ]{7,}\d"


def extract_entities(text):
    items = set()

    # DATE/TIME → spaCy
    doc = nlp_spacy(text)
    for ent in doc.ents:
        if ent.label_ in ["DATE", "TIME"]:
            items.add(ent.text)

    # PER/ORG/LOC → BABELSCAPE NER
    ner_out = ner_multi(text)
    for ent in ner_out:
        items.add(ent["word"])

    # Email
    for e in re.findall(email_regex, text):
        items.add(e)

    # Phone
    for p in re.findall(phone_regex, text):
        items.add(p)

    return list(items)


def highlight_markdown(text, entities):
    # sắp xếp theo độ dài để tránh conflict
    entities = sorted(entities, key=len, reverse=True)

    for e in entities:
        if e.strip() == "":
            continue

        safe_e = re.escape(e)
        text = re.sub(
            rf"(?<!\*)({safe_e})(?!\*)",
            r"**\1**",
            text
        )
    return text



# =====================================================
# FILE LOADING + CHUNKING (KHÔNG ĐỔI)
# =====================================================

def load_file_pages(path):
    ext = os.path.splitext(path)[1].lower()

    if ext == ".pdf":
        reader = PdfReader(path)
        pages = []
        for page in reader.pages:
            pages.append(page.extract_text() or "")
        return pages

    elif ext == ".docx":
        doc = Document(path)
        pages = []
        buffer = []
        cnt = 0

        for para in doc.paragraphs:
            t = para.text.strip()
            if t:
                buffer.append(t)
                cnt += 1

            if cnt >= 20:
                pages.append("\n".join(buffer))
                buffer = []
                cnt = 0

        if buffer:
            pages.append("\n".join(buffer))

        return pages

    elif ext == ".pptx":
        pres = Presentation(path)
        pages = []
        for slide in pres.slides:
            slide_text = []
            for shape in slide.shapes:
                if hasattr(shape, "text"):
                    slide_text.append(shape.text)
            pages.append("\n".join(slide_text))
        return pages

    else:
        raise ValueError("Unsupported file format")


def chunk_page(text, chunk_size=800, overlap=200):
    words = text.split()
    chunks = []
    start = 0

    while start < len(words):
        end = start + chunk_size
        chunks.append(" ".join(words[start:end]))
        start = end - overlap

    return chunks


def build_chunks(path):
    pages = load_file_pages(path)
    all_chunks, all_ids, all_meta = [], [], []

    for page_idx, text in enumerate(pages):
        page_num = page_idx + 1
        chunks = chunk_page(text)

        for ci, c in enumerate(chunks):
            all_chunks.append(c)
            all_ids.append(f"{os.path.basename(path)}_p{page_num}_c{ci}")
            all_meta.append({"page": page_num, "chunk": ci})

    return all_chunks, all_ids, all_meta, pages


# =====================================================
# EMBEDDING + VECTOR DB
# =====================================================

model = SentenceTransformer("Qwen/Qwen3-Embedding-0.6B")

chroma = chromadb.Client()
collection = chroma.get_or_create_collection(
    name="pdf_docs",
    metadata={"hnsw:space": "cosine"}
)


def index_pdf(path, batch_size=2):
    chunks, ids, metadata, pages = build_chunks(path)

    all_embeds = []

    for i in range(0, len(chunks), batch_size):
        batch = chunks[i:i+batch_size]
        with torch.no_grad():
            vec = model.encode(batch).tolist()
        all_embeds.extend(vec)

    collection.add(
        ids=ids,
        documents=chunks,
        metadatas=metadata,
        embeddings=all_embeds
    )

    return pages


def get_surrounding_pages(page, pages):
    prev = pages[page-2] if page > 1 else None
    this = pages[page-1]
    nextp = pages[page] if page < len(pages) else None
    return prev, this, nextp



# =====================================================
# SEARCH — THÊM HIGHLIGHT NER CHUNK
# =====================================================

def search(query, pages, top_k_first=3, top_k_second=5, chunk_size=800, overlap=200):

    # STEP 1 — query lần đầu
    q_emb = model.encode([query]).tolist()
    result = collection.query(
        query_embeddings=q_emb,
        n_results=top_k_first,
        include=["documents", "metadatas"]
    )
    metas = result["metadatas"][0]

    # STEP 2 — mở rộng trang
    expanded_pages = []
    expanded_page_ids = set()

    for meta in metas:
        page = meta["page"]
        prev, this, nextp = get_surrounding_pages(page, pages)
        for p in [prev, this, nextp]:
            if p and p not in expanded_page_ids:
                expanded_pages.append(p)
                expanded_page_ids.add(p)

    # STEP 3 — rechunk để rerank
    big_text = "\n\n".join(expanded_pages)
    words = big_text.split()

    re_chunks = []
    re_chunk_pages = []

    # tạo boundary theo expanded_pages
    page_boundaries = []
    offset = 0
    for idx, pg_text in enumerate(expanded_pages):
        length = len(pg_text.split())
        page_boundaries.append((offset, offset + length - 1, idx))
        offset += length

    start = 0
    while start < len(words):
        end = start + chunk_size
        chunk_words = words[start:end]
        chunk_text = " ".join(chunk_words)

        # xác định chunk thuộc page index nào
        page_idx = None
        for (s, e, pg_index) in page_boundaries:
            if not (end < s or start > e):     # overlap => same page
                page_idx = pg_index
                break

        re_chunks.append(chunk_text)
        re_chunk_pages.append(page_idx)

        start += chunk_size - overlap

    # STEP 3.5 — nếu 1 trang có >=2 chunk -> gộp luôn nguyên trang
    from collections import Counter
    count_page_chunks = Counter(re_chunk_pages)

    whole_pages_needed = {p for p, c in count_page_chunks.items() if c >= 2}

    if whole_pages_needed:
        # Lấy nguyên văn các trang được chọn
        re_chunks = []
        for pidx in whole_pages_needed:
            re_chunks.append(expanded_pages[pidx])

    # STEP 4 — rerank
    embeds = model.encode(re_chunks).tolist()

    temp = chromadb.Client().create_collection(
        name="temp_rerank",
        metadata={"hnsw:space": "cosine"},
        get_or_create=True
    )

    ids = [f"temp_{i}" for i in range(len(re_chunks))]
    temp.add(ids=ids, embeddings=embeds, documents=re_chunks)

    result2 = temp.query(
        query_embeddings=q_emb,
        n_results=1,
        include=["documents", "distances"]
    )

    best_chunk = result2["documents"][0][0]
    best_score = 1 - result2["distances"][0][0]

    # STEP 5 — GỘP TOÀN BỘ expanded page thành full text cuối cùng
    full_output_text = "\n\n".join(expanded_pages)

    # STEP 6 — extract full entities
    full_entities = extract_entities(full_output_text)

    # STEP 7 — highlight toàn văn bản
    highlighted_full = highlight_markdown(full_output_text, full_entities)

    # STEP 8 — trả ra gọn sạch
    return {
        "pages_used": list(expanded_page_ids),
        "score": best_score,
        "entities": full_entities,
        "highlighted_full_text": highlighted_full,
        "raw_full_text": full_output_text
    }




Device set to use cuda:0


In [8]:
pages = index_pdf("test3.pdf")

In [4]:
results = search("Quy định chuyển tiếp", pages,top_k_first=3, top_k_second=5, chunk_size=800, overlap=200)
for r in results:
            print("\n" + "="*80)
            print(f"Rank: {r['rank']}  |  Score: {r['score']:.4f}")
            print("-"*80)
            print(r["highlighted"])
            print("="*80)


Rank: 1  |  Score: 0.4336
--------------------------------------------------------------------------------
ban hành kèm theo Thông tư số 10/2009/TT -BGDĐT ngày 07 tháng 5 năm 2009 và được sửa đổi, bổ sung **môt** số điều theo Thông tư số 05/**2012**/TT - BGDĐT ngày 15 tháng 02 năm **2012** của **Bộ trưởng Bộ Giáo dục và Đào tạo**; - Đối với các khóa tuyển sinh kể từ thời điểm Thông tư này có hiệu lực thi hành đến hết ngày 31 tháng 12 năm 2018, quy định về tiêu chuẩn người hướng dẫn nghiên cứu sinh tại **trường Đại học Thủy lợi** (thuộc nhóm ngành II): là tác giả chính tối thiểu 01 báo cáo hoặc công trình khoa học đăng trong kỷ yếu hội thảo quốc tế có phản biện hoặc ít nhất một chương sách tham khảo có mã số chuẩn quốc tế ISBN do các nhà xuất bản nước ngoài phát hành hoặc 01 bài báo đăng trong tạp chí khoa học nước ngoài có phản biện thuộc lĩnh vực nghiên cứu khoa học liên quan đến đề tài luận án của nghiên cứu sinh; - Nghiên cứu sinh **trường Đại học Thủy lợi** (thực hiện những đề tài

In [9]:
search("Quy định chuyển tiếp", pages,top_k_first=3, top_k_second=5, chunk_size=800, overlap=200)

[{'page_id': 0,
  'page_text': 'Những điểm mới trong Quy chế tuyển sinh và đào tạo tiến sĩ \nNgày 04/4, Bộ Giáo dục và Đào tạo đã chính thức ban hành Quy chế tuyển sinh và đào \ntạo tiến sĩ. Quy chế có hiệu lực từ 18/5/2017 với nhiều điểm mới so với Quy chế ban hành \nnăm 2009 và sửa đổi bổ sung năm 2012 như sau: \n 1.  Số lần xét tuyển trong năm: Không hạn chế số lần xét tuyển trong năm (01 lần \nhoặc nhiều lần) điểm này khác với quy chế cũ chỉ xét 2 đợt/năm.  \n2. Đầu vào NCS được đặt ra với nhiều điều kiện cao hơn: \n- Theo quy định mới, người ứng tuyển vào NCS phải có bằng thạc sĩ hoặc tốt nghiệp \nđại học loại giỏi. Quy định hiện hành, người ứng tuyển vào NCS chỉ cần có bằng tốt nghiệp \nđại học loại khá; \n- Ứng viên NCS phải là tác giả 01 bài báo hoặc báo cáo liên quan đến lĩnh vực dự \nđịnh nghiên cứu đăng trên tạp chí khoa học hoặc kỷ yếu hội nghị, hội thảo khoa học chuyên \nngành có phản biện trong thời hạn 03 năm (36 tháng) tính đến ngày đăng ký dự tuyển; \n- Về trình độ ngo